# Ph3_instruct

> Experiments with loading and evaluating phi-3 instruct

## References

python -m mlx_vlm.generate --model mlx-community/Phi-3-vision-128k-instruct-8bit --max-tokens 100 --temp 0.0


## Microsoft Phi-3Cookbook

- [Microsoft Phi-3 Cookbook](https://github.com/microsoft/Phi-3CookBook/blob/3f19744fe79582fd85d858895a974a70b254d472/code/09.UpdateSamples/Aug/mlx-phi35-vision.ipynb)
- [https://medium.com/@researchgraph/what-is-phi-3-5-vision-101ce7c4d410#:~:text=One%20of%20Phi%2D3.5%2Dvision's,Character%20Recognition)%20and%20chart%20interpretation.](https://medium.com/@researchgraph/what-is-phi-3-5-vision-101ce7c4d410#:~:text=One%20of%20Phi%2D3.5%2Dvision's,Character%20Recognition)%20and%20chart%20interpretation.)


In [ ]:
#| default_exp phi3_instruct


In [ ]:
#| hide
from nbdev.showdoc import *

## Image loadin from command line

```sh
cvardema$ python -m mlx_vlm.generate --model mlx-community/Phi-3-vision-128k-instruct-8bit --max-tokens 100 --temp 0.0
Fetching 14 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 80881.90it/s]
/Users/cvardema/dev/git/nd-crane/vlm-experiments/.venv/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
==========
Image: http://images.cocodataset.org/val2017/000000039769.jpg 

Prompt: <|user|>
<|image_1|>
What are these?<|end|>
<|assistant|>

These are cats lying on a pink surface with remote controls.<|end|>
==========
Prompt: 4.995 tokens-per-sec
Generation: 19.464 tokens-per-sec

```

![Test Image](./img/000000039769.jpg)

In [ ]:
amimport mlx.core as mx
from mlx_vlm import load, generate
model_path = "mlx-community/Phi-3-vision-128k-instruct-8bit"
model, processor = load(model_path,processor_config={"trust_remote_code":"True"})

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
from PIL import Image
import requests, base64

image = Image.open("./img/000000039769.jpg")
placeholder = f"<|image_1|>\n"

In [ ]:
prompt = processor.tokenizer.apply_chat_template(
    [{"role": "user", "content": f"<image>\nWhat are these?"}],
    tokenize=False,
    add_generation_prompt=True,
)

In [ ]:
output = generate(model, processor, image, placeholder+prompt, verbose=False)
output

'These are cats lying on a pink surface with remotes beside them.<|end|>'

In [ ]:
messages = [
                {"role": "user", "content": "Summarize the video."}, 
]

In [ ]:
prompt = processor.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

In [ ]:
output = generate(model, processor, images, placeholder+prompt, verbose=False, max_tokens=1024)


## Document processing

### FAA Authorized Release Certificate
![](./img/authorized_release_certificate.png)



## Prompt for Document Analysis

# Analyze document page

```python
document_prompt = """
<|image_1|>
This image is a page from a scientific paper. Please analyze the content of this page, including:
1. The title of the paper (if visible)
2. Section headers
3. Key points or findings mentioned in the text
4. Any references to figures or charts
5. Any visible citations or references

Please be as detailed as possible in your analysis.
"""

document_conversation = [
    {"role": "user", "content": document_prompt}
]

document_response = generate_response(document_conversation, [document_page])
document_conversation.append({"role": "assistant", "content": document_response})
```

In [ ]:
image = Image.open("./img/authorized_release_certificate.png")
placeholder = f"<|image_1|>\n"

prompt = processor.tokenizer.apply_chat_template(
    [{"role": "user", "content": f"<image>\nWhat conceptual entities are contained in this document?"}],
    tokenize=False,
    add_generation_prompt=True,
)

In [ ]:
output = generate(model, processor, image, placeholder+prompt, verbose=False)
output

"The document contains the conceptual entities of 'Approving Civil Aviation', 'FAA Form 8130-3', 'AIRWOR', 'FAA Form 8130-3 (02-140)', and 'User/Installer Responsibilities'.<|end|>"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()